In [130]:
import re
from dataclasses import dataclass, field

@dataclass
class Valve:
    id: str
    flow: int
    connections: list[str]
    distance: dict[str, int] = field(default_factory=int)


def compute_distances_from(valves: dict[str,Valve], valve: Valve) -> dict[str,tuple[int, int]]:
    visited = set()
    queue = [(valve.id, 0)]
    res = {}
    while queue and (t := queue.pop(0)):
        next_valve, distance = t 
        res[next_valve] = min(res.get(next_valve, distance), distance)

        visited.add(next_valve)
        for v in valves[next_valve].connections:
            if v not in visited:
                queue.append((v, distance + 1))
    return res 

def parse():
    pattern = re.compile("Valve (?P<valve>[A-Z]{2}) has flow rate=(?P<flow>\d+); tunnels? leads? to valves? (?P<tunnels>.*)")
    retval = {}

    for line in open("input.txt"):
        res = pattern.match(line)
        connections = set(res.group("tunnels").split(", "))
        id = res.group("valve")
        retval[id] = Valve(id, int(res.group("flow")), connections)

    for v in retval.values():
        v.distance = compute_distances_from(retval, v)

    return retval

def bfs(valves: dict[str, Valve], valve: Valve) -> tuple[int, list[Valve]]:
    queue = []
    queue.append((0, 30, [("AA", 30, 0)], set([v for v in valves.keys() if valves[v].flow > 0])))
    
    best = -1
    best_path = []
    i = 0
    while queue:
        score, time, path, valves_to_open = queue.pop(0)
        if time < 0:
            continue

        if best < score:
            best = score
            best_path = path

        for v_id in valves_to_open:
            prev_valve = valves[path[0][0]]
            curr_valve = valves[v_id]
            distance = prev_valve.distance[v_id]
            
            curr_score = (time - distance - 1) * curr_valve.flow
            queue.append((score + curr_score, time - distance - 1,  [(v_id, time - distance - 1, curr_score)] + path, valves_to_open - set([v_id])))

        i += 1

    return best, best_path

def pretty_print(path: list[tuple[str, int]]):
    for valve, t, score in path:
        print("Turn valve", valve, "of at", t, score) 

valves = parse()




In [131]:
bfs(valves, valves["AA"])

(-1, [])